In [1]:
import asyncio
import random
import pandas as pd
import time
from dotenv import load_dotenv
# from mem0 import AsyncMemory
from names_generator import generate_name
from google import genai
from google.genai import types


In [ ]:
load_dotenv()

client = genai.Client()
# memory = AsyncMemory()

MODEL = "gemini-2.0-flash"

SYSTEM_INSTRUCTION = """
You are a character generator.
You will be told to create a character and will leverage the provided schema to create unique characters.
The user will provide you with the following prompt schema:

CREATE CHARACTER <RANDOM CODENAME> <ROLE>

Examples:

- CREATE CHARACTER "Vigorous Goldstein" "Commander"
- CREATE CHARACTER "Moby Miester" "Special Forces Operator"
- CREATE CHARACTER "Baby Howie" "Civilian"

You will then make up a character based on the codename and the provided schema; the first and last name will be made up by you.

The characters will be used in a game that is set in the not too distant future where players control a super soldier that will accomplish missions across various theaters of war across the globe.
Maintain a consistent style and realistic tone for the characters; ensure that the characters are consistent with the game setting and are not generic or linked to common tropes.
"""

ROLES = [
    "Commander",
    "Special Forces Operator",
    "Sniper",
    "Medic",
    "Engineer",
    "Technician",
    "Support",
    "Recon",
    "Ranger",
    "Rogue",
    "Assassin",
    "Mercenary",
    "Pilot",
    "Artillery Specialist",
    "Logistics Specialist",
    "Civilian"
]

SCHEMA_CONFIG = types.Schema(
    type='object',
    properties={
        'first_name': {
            'type': 'string',
            'description': 'The first name of the character.'
        },
        'last_name': {
            'type': 'string',
            'description': 'The last name of the character.'
        },
        'bio': {
            'type': 'string',
            'description': 'The bio of the character in around 100 words.'
        },
        'alignment': {
            'type': 'string',
            'enum': ['lawful good', 'neutral good', 'chaotic good', 'lawful neutral', 'true neutral', 'chaotic neutral', 'lawful evil', 'neutral evil', 'chaotic evil'],
            'description': 'The alignment of the character.'
        },
        'personality': {
            'type': 'array',
            'min_items': 3,
            'max_items': 5,
            'items': {
                'type': 'string',
                'description': 'A personlaity trait of the character (e.g. "brave", "cautious", "greedy", "selfish", "kind").'
            },
            'description': 'The personality of the character; this is a list of personality traits that the character possesses.'
        },
        'interests': {
            'type': 'array',
            'min_items': 3,
            'max_items': 5,
            'items': {
                'type': 'string',
                'description': 'An interest of the character (e.g. "fishing", "hunting", "crafting", "cooking", "reading").'
            },
            'description': 'The interests of the character; this is a list of interests that the character possesses.'
        },
        'goals': {
            'type': 'array',
            'min_items': 3,
            'max_items': 5,
            'items': {
                'type': 'string',
                'description': 'A goal of the character (e.g. "hunt a great beast", "find a lost relic", "root out all evil").'
            },
            'description': 'The goals of the character; this is a list of goals that the character possesses.'
        },
        'gender': {
            'type': 'string',
            'enum': ['male', 'female', 'other'],
            'description': 'The gender of the character.'
        },
        'age': {
            'type': 'integer',
            'description': 'The age of the character.'
        },
        'height': {
            'type': 'number',
            'description': 'The height of the character in centimeters.'
        },
        'weight': {
            'type': 'number',
            'description': 'The weight of the character in pounds.'
        },
        'health': {
            'type': 'integer',
            'description': 'The health points of the character.'
        },
        'stamina': {
            'type': 'integer',
            'description': 'The stamina points of the character.'
        },
        'tech_points': {
            'type': 'integer',
            'description': 'The tech points of the character.'
        },
        'attack': {
            'type': 'integer',
            'description': 'The attack points of the character.'
        },
        'defense': {
            'type': 'integer',
            'description': 'The defense points of the character.'
        },
        'level': {
            'type': 'integer',
            'description': "The level of the character; this is the level of the character in the game. Ensure it's scaled to the character's stats."
        },
        'experience': {
            'type': 'integer',
            'description': 'The experience points of the character in accordance to the character\'s level.'
        },
    },
    required=['first_name', 'last_name', 'bio', 'alignment', 'personality', 'interests', 'goals', 'gender', 'age', 'height', 'weight', 'health', 'stamina', 'tech_points', 'attack', 'defense', 'level', 'experience'],
    property_ordering=['first_name', 'last_name', 'bio', 'alignment', 'personality', 'interests', 'goals', 'gender', 'age', 'height', 'weight', 'health', 'stamina', 'tech_points', 'attack', 'defense', 'level', 'experience']
)

In [ ]:
character_generator = client.aio.chats.create(
    model=MODEL,
    config=types.GenerateContentConfig(
        temperature=0.7,
        system_instruction=SYSTEM_INSTRUCTION,
        response_mime_type='application/json',
        response_schema=SCHEMA_CONFIG
    )
)

In [ ]:
async def generate_character(contents: str) -> dict:
    seed = random.randint(0, 1000000)
    try:
        response = await client.aio.models.generate_content(
            model=MODEL,
            config=types.GenerateContentConfig(
                temperature=0.75,
                seed=seed,
                system_instruction=SYSTEM_INSTRUCTION,
                response_mime_type='application/json',
                response_schema=SCHEMA_CONFIG
            ),
            contents=contents
        )
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

    return response


In [7]:
response = await generate_character(f"CREATE CHARACTER {generate_name(style='capital')} {random.choice(ROLES)}")
response.parsed

{'first_name': 'Isabelle',
 'last_name': 'Reyes',
 'bio': 'Isabelle "Recursing Rhodes" Reyes is a highly skilled assassin known for her methodical approach and adaptability. Growing up in the slums of Neo-Manila, she learned to survive by any means necessary. Recruited by a clandestine organization at a young age, she underwent rigorous training in various combat disciplines and espionage tactics. Her ability to analyze and exploit vulnerabilities makes her a formidable opponent. Despite her deadly profession, Isabelle harbors a strong desire for a peaceful life, a stark contrast to the violence she perpetrates.',
 'alignment': 'chaotic neutral',
 'personality': ['calculating',
  'resourceful',
  'stoic',
  'independent',
  'secretive'],
 'interests': ['urban exploration',
  'cybernetics',
  'knife collecting',
  'meditation',
  'linguistics'],
 'goals': ['complete assigned contracts efficiently',
  'accumulate enough wealth to retire',
  'discover the truth behind her recruitment',
  

In [7]:
random.randint(0, 100)

68